# Introduction - cid_mosaic

Welcome!

This notebook is intended as a preparation for the second workshop and it introduces a custom made Python class designed to help you analyze the simulation results from Eclipse MOSAIC.

It can also be used to programmatically interact with the federate configurations.

Make sure everything works before attending the workshop.

For questions, troubleshooting and bugs please reach me (Ongun Türkcüoglu) via email: ongun.turkcuoglu@campus.tu-berlin.de 

Lets start by importing several modules we will need for our analysis!

Hint: You can display the documentation by highlighting the object/method and pressing Shift+Tab

In [ ]:
%matplotlib inline

from cid_mosaic import cid_mosaic
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Init seaborn for pretty plots
sns.set()

# Larger plots
matplotlib.rcParams['figure.figsize'] = [15, 5]

Create an object `cid_mosaic()` and initialize the object using the path to Eclipse MOSAIC and simulation name, i.e. `'/home/onqi/Documents/eclipse_mosaic/'` and `'Tiergarten'`.

Third parameter is a boolean value, which should be `True` if you have a UNIX system and `False` is you have Windows running.

**Hint:** Don't forget to escape the backslashes if you have a Windows, i.e.`'C:\\Users\\ongun\\Desktop\\eclipse-mosaic'`

In [ ]:
mosaic = cid_mosaic('C:\\Users\\ongun\\Desktop\\eclipse-mosaic', 'Tiergarten', False)

The method `run_simulation()` will run the selected simulation and display the terminal output in the notebook.

If you are not using a Jupyter notebook, set the boolean parameter to `False`

In [ ]:
!{mosaic.run_simulation(jupyter=True)}

Now that we ran the simulation, we can access the generateed output. Normally, this file is in your Eclipse MOSAIC folder under `logs/log-{yyyymmdd}-{hhmmss}-{simulation-name}`

However, you can simply call the method `select_simulation_result(idx=0)` to access and generate pandas DataFrame of your latest simulation.

Hint: idx=0 is the most recent simulation, idx:1..n is previous simulations, with ascending integers corresponding to older simulations. 

In [ ]:
mosaic.select_simulation_result(idx=0)

## Working with MOSAIC DataFrames

After you select the simulation results using `select_simulation_result()`, a DataFrame is generated, which holds every single log event that took place during simulation runtime.

However, not all data is interesting, especially if you are looking at unique vehicles or want to plot only certain traffic lights and road side units for your reports. There are three different calls you can make to find out which data you need/want/find interesting.

In [ ]:
mosaic.get_df_events

`get_df_events` returns a list of the events registered by the simulator. For example, a vehicle will register a `VEHICLE_UPDATES` event every time the application runs, and at each sampling instant, it will log certain values, for example, speed, position, heading, etc.

In [ ]:
mosaic.get_df_labels('VEHICLE_UPDATES')

`get_df_labels` gives you the labels of the generated DataFrame. In pandas, these labels are called **Fields** and they label the columns.

In [ ]:
mosaic.get_df_apps

`get_df_apps` returns a list of all the executed applications during runtime. If you need, for example, the vehicle with the index 0, you'd need the value `veh_0`. Other possible values are (in this case) `rsu_0, tl_0, tl_1, tl_2, veh_1, veh_2` or `veh_3`

### Filtering pd.DataFrame

Now that you know how to investigate the simulation output, lets filter some data using numpy and pandas: `cid_mosaic` has a method called `df2np()`, which is just a wrapper for `np.asfarray()`.

Imagine you want to take a look at the speed of each vehicle for this simulation. You investigated the data and found out that there are four vehicles `veh_0`, `veh_1`, `veh_2` and `veh_3`. Since we want the speeds we are also going to need our event `VEHICLE_UPDATES`.

We also need a figure out which fields we are going to need. In this specific case, we can extract every field using `'all'` since there is only 7 of them, but in larger data sets you might want to specify which fields you need. For example: `mosaic.filter_df('VEHICLE_UPDATES', 'veh_0', 'Speed')` for speed of vehicle 0.

Hint: `Event`, `Time` and `Name` (application name) are always extracted and you don't need to specify these again.

In [ ]:
veh0 = mosaic.filter_df(Event='VEHICLE_UPDATES', Name='veh_0', select='all')
veh1 = mosaic.filter_df(Event='VEHICLE_UPDATES', Name='veh_1', select='all')
veh2 = mosaic.filter_df(Event='VEHICLE_UPDATES', Name='veh_2', select='all')
veh3 = mosaic.filter_df(Event='VEHICLE_UPDATES', Name='veh_3', select='all')

Our filtered variables `veh_0`, `veh_1`, `veh_2` and `veh_3` are still DataFrames, which means we can easily access the fields as if we would access the methods of an object:

`veh0.Time` is the sampling instants, `veh1.Speed` is the speed of the vehicle 1, etc.

Lets plot the speeds and the sampled GPS locations of our four vehicles!

In [ ]:
plt.plot(veh0.Time, veh0.Speed, label='veh0')
plt.plot(veh1.Time, veh1.Speed, label='veh1')
plt.plot(veh2.Time, veh2.Speed, label='veh2')
plt.plot(veh3.Time, veh3.Speed, label='veh3')
plt.title('Speed of Vehicle 0-1-2-3')
plt.legend()

In [ ]:
plt.figure()
plt.scatter(np.asfarray(veh0.PositionLongitude),
            np.asfarray(veh0.PositionLatitude),
            linewidths=0.5,
            label='veh0')
plt.scatter(np.asfarray(veh1.PositionLongitude),
            np.asfarray(veh1.PositionLatitude),
            linewidths=0.5,
            label='veh1')
plt.scatter(veh2.PositionLongitude.astype(float),
            veh2.PositionLatitude.astype(float),
            linewidths=0.5,
            label='veh2')
plt.scatter(veh3.PositionLongitude.astype(float),
            veh3.PositionLatitude.astype(float),
            linewidths=0.5,
            label='veh3')
plt.title('Road Travelled - Vehicle 0-1-2-3')
plt.legend()

Note that for the plotting of the GPS coordinates, a type conversion is necessary. The coordinates are read as type `object` but need to be of type `float` to be properly plotted. You can achieve this by either calling the method `.astype(float)` (see veh2 & 3 above) or wrapping the data in `np.asfarray()` (see veh0 & 1) to convert them to a numpy array.

### Configuring the Federates

Eclipse MOSAIC is designed to be extendable using the concept of federates. This will be explained more in detail during the workshop lecture, but simply put, the federates communicate with the main application and affect the behaviour of the vehicles.

Some examples of federates are `cell`, `ns3`, `omnetpp`, `environment`, `application`, `sns` and `sumo`, among others.

Federates are configured using JSON files, which you can very well just edit using any text editor without much hassle. However, we implemented an interface which can be programmatically used to edit and configure the federates.

Hint: It is highly recommended to backup the whole simulation folder before you play with any federate configurations, since it is really easy to type an invalid value and lose track of what you have done. In the worst case scenario, you can restore the simulation easily and start over.

There are two calls and two main methods: `get_federates`, `pprint_curr_fed`, `retrieve_federate(federate, idx)` and `set_federate_value(tree, value)`

In [ ]:
mosaic.get_federates

`get_federates` returns a list of all available federates, including `scenario_config.json`. The latter JSON file is not exactly a federate, but still, it is a json file which can be configured with our method.

`retrieve_federate(federate, idx)` is a method which allows you to explore the federate folder, if idx is None, or it retrieves the federate configuration and load it up.

Lets explore `cell` federate:

In [ ]:
mosaic.retrieve_federate('cell')

From previous experience, I know that `network.json` is the configuration for the cellular network, so lets retrieve that using `idx=1`

In [ ]:
mosaic.retrieve_federate('cell', idx=1)

The full path of the retrieved federate is displayed and loaded in the background. Now you can explore the federate configuration using `pprint_curr_fed`

In [ ]:
mosaic.pprint_curr_fed

A JSON file, for all intents and purposes, acts like a dictionary when loaded in Python.

Using the method `set_federate_value(tree, value)` will set the selected key to the desired value.

The tree is seperated using dot (`.`) as we descend further down the line, for example `"globalNetwork.uplink.delay.expDelay"`

Depending on the tree structure within the json file, the syntax could differ, i.e. `spec = ('events', ['location.area.a'])`.

The current value of `"globalNetwork.uplink.delay.expDelay"` is `"150 ms"`. Notice the quotes and make sure you abide by the type of the existing value.

Lets set this key to `"250 ms"`.

In [ ]:
mosaic.set_federate_value('globalNetwork.uplink.delay.expDelay', '250 ms')

And print our new federate configuration once again to validate the change.

In [ ]:
mosaic.pprint_curr_fed

The new value of `"globalNetwork.uplink.delay.expDelay"` is `"250 ms"`.

Make sure to run the simulation again to observe what kind of changes happen.

In [ ]:
!{mosaic.run_simulation(jupyter=True)}

Thank you for following the introduction and don't forget to play with the tool a little bit more to get used to it. 

Don't forget to report any problems or bugs you might find! We'd be happy to work on it with you!

**See you in the workshop!**